In [ ]:
#@title Import libraries
!git clone https://github.com/SukritJaidee/pred_temp.git 
%cd /content/pred_temp

!pip install -q meteostat
!pip install -q mercantile
!pip install -q mpmath

import time
import pytz
import warnings
# import schedule
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# from pycaret.regression import *
from datetime import datetime, timedelta, date, tzinfo
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from pred_temp.x6_meteo import open_meteo_v1
from pred_temp.x3_ibm_api import x3_api
from pred_temp.tmd_api import tmd_weather
from pred_temp.x4_weather_api import x4_api
from pred_temp.loc_level10 import get_level10_pos
from pred_temp.x1_station_meteo_api import x1_api
from pred_temp.x2_point_meteo_api import x2_api, x1_station_x2

path_save = '/content/drive/MyDrive/result_weather/' 
path = '/content/drive/MyDrive/chula_weather/one_model/keras_model/'
root_path =  '/content/drive/MyDrive/chula_weather/one_model/last_dataset/'
df = get_level10_pos()

In [3]:
#@title Model

# def ann_model(X_train):
#       layer = tf.keras.layers.experimental.preprocessing.Normalization()
#       layer.adapt(X_train)
#       model = keras.Sequential([layer,
#                                             keras.layers.Dense(64, activation='relu', kernel_initializer='normal', kernel_regularizer="l2", input_shape=(68,)),
#                                             keras.layers.Dropout(0.2),
#                                             keras.layers.Dense(32, activation='relu', kernel_initializer='normal', kernel_regularizer="l2"),
#                                             keras.layers.Dropout(0.2),
#                                             keras.layers.Dense(1, activation='relu')
#                                             ])
#       loss = tf.keras.losses.MeanSquaredError(name="mse")
#       optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
#       m1 = tf.keras.metrics.RootMeanSquaredError()
#       m2 = tf.keras.metrics.MeanAbsoluteError()
#       model.compile( optimizer=optimizer,  loss = loss, metrics = [m1, m2])
#       return model

def ann_model(X_train):
      input_shape = X_train.shape[1:]
      layer = tf.keras.layers.experimental.preprocessing.Normalization()
      layer.adapt(X_train)
      model = keras.Sequential([layer,
                                            keras.layers.Dense(64, activation='relu', kernel_initializer='normal', kernel_regularizer="l2", input_shape=(input_shape,)),
                                            keras.layers.Dropout(0.2),
                                            keras.layers.Dense(32, activation='relu', kernel_initializer='normal', kernel_regularizer="l2"),
                                            keras.layers.Dropout(0.2),
                                            keras.layers.Dense(1, activation='relu')
                                            ])
      loss = tf.keras.losses.MeanSquaredError(name="mse")
      optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
      m1 = tf.keras.metrics.RootMeanSquaredError()
      m2 = tf.keras.metrics.MeanAbsoluteError()
      model.compile( optimizer=optimizer,  loss = loss, metrics = [m1, m2])
      return model

In [ ]:
#@title Model_v0 : w IBM
# df_po = pd.read_csv(root_path+'final_data.csv')
# x, y = df_po.drop("temperature", axis = 1), df_po[["temperature"]]
# X_train, X_test, y_train, y_test = train_test_split(x, y,  test_size=0.2, shuffle=True)
# models = 'keras_best_model_rmse_0528_mse_04208.h5'
# saved_lr = ann_model(X_train)
# saved_lr.load_weights(path+models)

# cols = ['lat', 'lon', 'latitude', 'longitude', 'elevation', 'distance', 'temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'year', 'month', 'day', 'hour', 'minute', 'temp.1', 'dwpt.1', 'rhum.1', 'wdir.1', 'wspd.1',
#                   'pres', 'lat.1', 'lon.1', 'year.1', 'month.1', 'day.1', 'hour.1', 'minute.1', 'requestedLatitude', 'requestedLongitude', 'latitude.1', 'longitude.1', 'precip1Hour', 'precip6Hour', 'precip24Hour', 'pressureChange',
#                   'pressureMeanSeaLevel', 'relativeHumidity', 'temperature', 'temperatureChange24Hour', 'temperatureMax24Hour', 'temperatureMin24Hour', 'temperatureDewPoint', 'temperatureFeelsLike', 'uvIndex',
#                   'visibility', 'windDirection', 'windSpeed', 'lat.2', 'lon.2', 'lats', 'lons', 'temp_cs', 'wind_kphs', 'wind_degrees', 'wind_dirs', 'pressure_mbs', 'precip_mms', 'humiditys', 'clouds', 'feelslike_cs',
#                   'windchill_cs', 'heatindex_cs', 'dewpoint_cs', 'will_it_rains', 'chance_of_rains', 'vis_kms', 'gust_kphs']

# def cal_from_source(q1, now_datetime, format_date, saved_lr):
#     #### x0
#     diff_time, hour, duration = 0, 0, 24
#     day = (datetime.now(pytz.timezone('Asia/Bangkok'))+timedelta(diff_time)).strftime("%Y-%m-%d")
#     x0 = tmd_weather(q1[0], q1[1], day, hour, duration)
#     #### x1
#     start_date, end_date = -1, 1
#     start = pd.to_datetime((now_datetime+timedelta(start_date)).strftime(format_date))
#     end = pd.to_datetime((now_datetime+timedelta(end_date)).strftime(format_date))
#     x1 = x1_api(q1, start, end)
#     #### x2
#     x2 = x2_api(q1, start, end)
#     if x2.shape[0] == 0:
#       x2 = x1_station_x2(q1, start=-1, end=1)
#       x2.drop(['latitude', 'longitude', 'elevation', 'distance'], axis=1, inplace=True)
#       x2 = x2.reindex(columns=['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'minute'])
#     #### x3
#     apikey =  "2601b3bc3f684d2681b3bc3f687d26f8"
#     format_date, start_date, end_date  = "%Y-%m-%d", 0, 1
#     start = pd.to_datetime(now_datetime+timedelta(start_date)).strftime(format_date)
#     end = pd.to_datetime(now_datetime+timedelta(end_date)).strftime(format_date)
#     x3 = x3_api(q1, start, end, apikey)
#     #### x4
#     x4  = x4_api(q1, start, end)
#     res = pd.concat([x0, x1, x2, x3, x4], axis=1,  join="inner")
#     res  = res.iloc[-1:,:]
#     return res

# def deploy_schedule(df, now_datetime, format_date, saved_lr, start_cp):
#     k = 0
#     for  i in tqdm(range(len(df))):
#         lat, lon = df['lat_south'][i], df['lon_east'][i]
#         q1 = (lat, lon)
#         try:
#             if k == 0:
#               res_update = cal_from_source(q1, now_datetime, format_date, saved_lr)
#             else:
#               res = cal_from_source(q1, now_datetime, format_date, saved_lr)
#               res_update = pd.concat([res_update, res], axis=0)
#             k+=1
#         except:
#             try:
#                 res_update_v, res_v = res_update.values, res.values
#                 cols_raw = res_update.columns
#                 res_update = np.concatenate((res_update_v, res_v), axis=0)
#                 res_update = pd.DataFrame(res_update, columns=cols_raw)
#             except:
#                 print(f'error i ={i}, lat = {round(lat,2)}, lon = {round(lon,2)}')

#     res1 = res_update.drop(['tc', 'rh', 'datatime'], axis=1)
#     res1.columns = cols

#     res_temp = res1['temperature']
#     res_temp.reset_index(drop=True, inplace=True)
#     res1 = res1.drop(['temperature'], axis=1)

#     df_loc = res1[['lat', 'lon']]
#     df_loc.reset_index(drop=True, inplace=True)

#     try:
#         res1 = res1.values.astype(np.float32)
#         y_pred = saved_lr.predict(res1, verbose=0)
#         y_pred = pd.DataFrame(y_pred, columns = ['ypred'])
#     except:
#         res1 = np.asarray(res1.values).astype(np.float32)
#         y_pred = saved_lr.predict(res1,verbose=0)
#         y_pred = pd.DataFrame(y_pred, columns = ['ypred'])

#     actual = res_update[['tc', 'datatime']]
#     actual.reset_index(drop=True, inplace=True)

#     res2 = pd.concat([y_pred, actual], axis=1,  join="inner")

#     df_val = res2[['ypred', 'tc', 'temperature', 'datatime']]
#     result = pd.concat([df_loc, df_val], axis=1,  join="inner")

#     success = True
#     end_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
#     delta = end_cp - start_cp
#     min = (delta.total_seconds()) / 60
#     print('minutes:', min)
#     start_cp, end_cp = start_cp.strftime("%d_%m_%Y_%H_%M"), end_cp.strftime("%d_%m_%Y_%H_%M")
#     info = pd.DataFrame({'success':[str(success)], 'datetime':[str(start_cp)], 'start_compute':[str(start_cp)], 'end_compute':[str(end_cp)]})
#     info.to_csv(path_save+'data/v4/'+'data_'+start_cp+'_info_delta.csv' ,encoding='utf-8-sig')
#     result.to_csv(path_save+'data/v4/'+'data_'+start_cp+'_data_delta.csv' ,encoding='utf-8-sig')
#     return True

# def job_B():
#     format_date = "%Y-%m-%d %H:%M:%S"
#     now_datetime = datetime.now(pytz.timezone('Asia/Bangkok'))
#     start_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
#     print("I'm working %s"%(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
#     success_r = deploy_schedule(df, now_datetime, format_date, saved_lr, start_cp)
#     print(success_r)
#     return success_r

# format_save = "%d_%m_%Y_%H_%M"
# prev_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
# cur_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
# while True:
#   try:
#     cur_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
#     if (cur_time[-2:] == '00') and ( int(cur_time[-5:-3]) - int(prev_time[-5:-3]) == 1):
#       success_r = job_B()
#       prev_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
#       print(prev_time)
#   except:
#     print('Error')

In [ ]:
#@title Model_v1 w/o IBM
df_po = pd.read_csv(root_path+'data_pmwa.csv')
x, y = df_po.drop("temp_2m", axis = 1), df_po[["temp_2m"]]
x.drop(['wspeed_100m', 'wdirection_100m', 'soil_temp_0_to_7cm', 'soil_temp_7_to_28cm'], axis =1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(x, y,  test_size=0.2, shuffle=True)
models = 'keras_best_model2_v1.h5'
model = ann_model(X_train)
model.load_weights(path+models)

cols =  [
              'temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'minute',  #x2 point meteostat
              'temp_2m', 'rh_2m', 'dewpoint_2m', 'apparent_temp', 'rain', 'cloudcover', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high', #x6 meteo
              'shortwave_rad', 'direct_rad', 'diffuse_rad', 'direct_normal_irr', 'wspeed_10m',  'wdirection_10m', 'wgusts_10m', 
              'lat.1', 'lon.1', 'elevation', 'lat_res', 
              'lon_res', 'lats', 'lons', 'temp_cs', 'wind_kphs', 'wind_degrees', 'wind_dirs', 'pressure_mbs', 'precip_mms', 'humiditys', #x4 weatherapi
             'clouds', 'feelslike_cs', 'windchill_cs', 'heatindex_cs', 'dewpoint_cs', 'will_it_rains', 'chance_of_rains', 'vis_kms', 'gust_kphs' #x4 weatherapi
              ]

def cal_from_source(q1, now_datetime, format_date, model):
        #### x0 tmd
        diff_time, hour, duration = 0, 0, 24
        day = (datetime.now(pytz.timezone('Asia/Bangkok'))+timedelta(diff_time)).strftime("%Y-%m-%d")
        x0 = tmd_weather(q1[0], q1[1], day, hour, duration)
        #### x2 point meteostat
        start_date, end_date = -1, 1
        start = pd.to_datetime((now_datetime+timedelta(start_date)).strftime(format_date)) 
        end = pd.to_datetime((now_datetime+timedelta(end_date)).strftime(format_date))
        x2 = x2_api(q1, start, end)
        if x2.shape[0] == 0:
          x2 = x1_station_x2(q1, start=-1, end=0)
          x2.drop(['latitude', 'longitude', 'elevation', 'distance'], axis=1, inplace=True)
          x2 = x2.reindex(columns=['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres', 'lat', 'lon', 'year', 'month', 'day', 'hour', 'minute'])
        #### x6 meteo
        format_date, start_date, end_date = "%Y-%m-%d", 0, 0
        start = pd.to_datetime(now_datetime+timedelta(start_date)).strftime(format_date)
        end = pd.to_datetime(now_datetime+timedelta(end_date)).strftime(format_date)
        x6 = open_meteo_v1(q1[0], q1[1], start, end)
        x6.index = x6['datetime1']
        x6.drop(['indexs', 'time', 'datetime1'], axis =1, inplace=True)
        #### x4 weatherapi
        format_date, start_date, end_date = "%Y-%m-%d", -1, 1
        start = pd.to_datetime(now_datetime+timedelta(start_date)).strftime(format_date)
        end = pd.to_datetime(now_datetime+timedelta(end_date)).strftime(format_date)
        x4  = x4_api(q1, start, end)
        res = pd.concat([x2, x6, x4], axis=1,  join="inner")
        res  = res.iloc[-1:,:]
        return res, x0

def deploy_schedule(df, now_datetime, format_date, model, start_cp):
      k = 0
      for  i in tqdm(range(len(df))):
          lat, lon = df['lat_south'][i], df['lon_east'][i]
          q1 = (lat, lon)
          try:
              if k == 0:
                res_update, x0_update = cal_from_source(q1, now_datetime, format_date, model)       
              else:
                res, x0 = cal_from_source(q1, now_datetime, format_date, model)
                res_update = pd.concat([res_update, res], axis=0)
                x0_update = pd.concat([x0_update, x0], axis=0)
              k+=1
          except Exception as e:
              print(e)
              try:
                  res_update_v, res_v = res_update.values, res.values
                  cols_raw = res_update.columns
                  res_update = np.concatenate((res_update_v, res_v), axis=0)
                  res_update = pd.DataFrame(res_update, columns=cols_raw)
              except Exception as e:
                  print(e)
                  print(f'error i ={i}, lat = {round(lat,2)}, lon = {round(lon,2)}')

      res_update.columns = cols
      temp_po = res_update['temp_2m']
      temp_po.reset_index(drop=True, inplace=True)
      res = res_update.drop(['temp_2m'], axis=1)

      df_loc = res[['lat', 'lon']]
      df_loc.reset_index(drop=True, inplace=True)

      try:
          res1 = res.values.astype(np.float32)
          y_pred = model.predict(res1)
          y_pred = pd.DataFrame(y_pred, columns = ['ypred'])
      except Exception as e:
          res1 = np.asarray(res.values).astype(np.float32)
          y_pred = model.predict(res1)
          y_pred = pd.DataFrame(y_pred, columns = ['ypred'])
          print(e)

      actual = x0_update[['tc', 'datatime']]
      actual.reset_index(drop=True, inplace=True)
      
      res2 = pd.concat([y_pred, actual], axis=1,  join="inner")
      res2['temperature'] = temp_po

      df_val = res2[['ypred', 'tc', 'temperature', 'datatime']]
      result = pd.concat([df_loc, df_val], axis=1,  join="inner")

      success = True
      end_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
      delta = end_cp - start_cp
      min = (delta.total_seconds()) / 60
      print('minutes:', min)
      start_cp, end_cp = start_cp.strftime("%d_%m_%Y_%H_%M"), end_cp.strftime("%d_%m_%Y_%H_%M")
      info = pd.DataFrame({'success':[str(success)], 'datetime':[str(start_cp)], 'start_compute':[str(start_cp)], 'end_compute':[str(end_cp)]})
      info.to_csv(path_save+ 'data/'+'data_'+start_cp+'_info_BlockingScheduler.csv' ,encoding='utf-8-sig')
      result.to_csv(path_save+'data/'+'data_'+start_cp+'_data_BlockingScheduler.csv' ,encoding='utf-8-sig')
      return True, result
      
def job_B():
    format_date = "%Y-%m-%d %H:%M:%S"
    now_datetime = datetime.now(pytz.timezone('Asia/Bangkok'))
    start_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
    print("I'm working %s"%(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    success_r = deploy_schedule(df, now_datetime, format_date, model, start_cp)
    print(success_r)
    return success_r

format_save = "%d_%m_%Y_%H_%M"
prev_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
cur_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
while True:
  try:
    cur_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
    if (cur_time[-2:] == '00') and ( int(cur_time[-5:-3]) - int(prev_time[-5:-3]) == 1):
      success_r = job_B()
      prev_time = datetime.now(pytz.timezone('Asia/Bangkok')).strftime(format_save) 
      print(prev_time)
  except:
    print('Error')

  0%|          | 0/25 [00:00<?, ?it/s]

(1100, 6)
   lon_west  lat_north  lat_south   lon_east  x_tiles  y_tiles
0  97.03125  20.632784  20.303418  97.382812      788      452
I'm working 2023-02-21 13:00:00


 10%|▉         | 106/1100 [22:26<85:42:47, 310.43s/it]

('Connection aborted.', TimeoutError(110, 'Connection timed out'))


 40%|███▉      | 437/1100 [54:56<57:00:51, 309.58s/it]

('Connection aborted.', TimeoutError(110, 'Connection timed out'))


 89%|████████▊ | 975/1100 [1:21:08<05:29,  2.64s/it]

In [ ]:
#@title Backup

# !pip install -r requirements_colab_r1.txt

# !pip install scikit-learn==1.2.0 #[1.0.1, 1.0.2, 0.23.2]
# !pip install APScheduler
# !pip install apscheduler

# !pip install --pre pycaret
# !pip install git+https://github.com/pycaret/pycaret.git#egg=pycaret

# models = 'Final Blend Model 20112022_950_ibm'
# saved_lr = load_model(models)

# unseen_pred = predict_model(saved_lr, data=res1)

# res2 = pd.concat([unseen_pred, actual], axis=1,  join="inner")

# df_val = res2[['prediction_label', 'tc', 'temperature', 'datatime']]

In [ ]:
#@title Backup1
# k = 0
# saved_lr = ann_model(X_train)
# saved_lr.load_weights(path+models)
# format_date = "%Y-%m-%d %H:%M:%S"
# now_datetime = datetime.now(pytz.timezone('Asia/Bangkok'))

# for  i in tqdm(range(len(df))):
#     lat, lon = df['lat_south'][i], df['lon_east'][i]
#     q1 = (lat, lon)
#     try:
#         if k == 0:
#           res_update = cal_from_source(q1, now_datetime, format_date, saved_lr)
#         else:
#           res = cal_from_source(q1, now_datetime, format_date, saved_lr)
#           res_update = pd.concat([res_update, res], axis=0)
#         k+=1
#     except:
#         try:
#             res_update_v, res_v = res_update.values, res.values
#             cols_raw = res_update.columns
#             res_update = np.concatenate((res_update_v, res_v), axis=0)
#             res_update = pd.DataFrame(res_update, columns=cols_raw)
#         except:
#             print(f'error i ={i}, lat = {round(lat,2)}, lon = {round(lon,2)}')

# res1 = res_update.drop(['tc', 'rh', 'datatime'], axis=1)
# res1.columns = cols

# res_temp = res1['temperature']
# res_temp.reset_index(drop=True, inplace=True)

# res1 = res1.drop(['temperature'], axis=1)

# df_loc = res1[['lat', 'lon']]
# df_loc.reset_index(drop=True, inplace=True)

# print(type(res1))
# try:
#     res1 = res1.values.astype(np.float32)
#     print(type(res1))
#     y_pred = saved_lr.predict(res1, verbose=0)
#     y_pred = pd.DataFrame(y_pred, columns = ['ypred'])
# except:
#     res1 = np.asarray(res1.values).astype(np.float32)
#     print(type(res1))
#     y_pred = saved_lr.predict(res1,verbose=0)
#     y_pred = pd.DataFrame(y_pred, columns = ['ypred'])

# actual = res_update[['tc', 'datatime']]
# actual.reset_index(drop=True, inplace=True)

# res2 = pd.concat([y_pred, actual], axis=1,  join="inner")
# res2['temperature'] = res_temp

# df_val = res2[['ypred', 'tc', 'temperature', 'datatime']]
# result = pd.concat([df_loc, df_val], axis=1,  join="inner")

# success = True
# start_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
# end_cp = datetime.now(pytz.timezone('Asia/Bangkok'))
# delta = end_cp - start_cp
# min = (delta.total_seconds()) / 60
# print('minutes:', min)
# start_cp, end_cp = start_cp.strftime("%d_%m_%Y_%H_%M"), end_cp.strftime("%d_%m_%Y_%H_%M")
# info = pd.DataFrame({'success':[str(success)], 'datetime':[str(start_cp)], 'start_compute':[str(start_cp)], 'end_compute':[str(end_cp)]})
# info.to_csv(path_save+'data/v4/'+'data_'+start_cp+'_info_delta.csv' ,encoding='utf-8-sig')
# result.to_csv(path_save+'data/v4/'+'data_'+start_cp+'_data_delta.csv' ,encoding='utf-8-sig')